In [3]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori

### Read dataset


In [4]:
sales_reciepts = pd.read_csv("dataset/201904 sales reciepts.csv")
sales_reciepts.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [5]:
product = pd.read_csv("dataset/product.csv")
product.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


### Data Wrangling

#### Merge data

In [6]:
sales_reciepts = sales_reciepts[["transaction_id", "transaction_date", "sales_outlet_id", "customer_id", "product_id", "quantity"]]
product = product[["product_id", "product_category", "product"]]

dataset = pd.merge(sales_reciepts, product, on="product_id", how="left")
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


#### Remove sizes

In [7]:
dataset[dataset["product"].str.contains("Dark chocolate")]["product"].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [8]:
dataset["product"].nunique()

80

In [9]:
dataset["product"] = dataset["product"].str.replace("Rg", "")
dataset["product"] = dataset["product"].str.replace("Sm", "")
dataset["product"] = dataset["product"].str.replace("Lg", "")

In [10]:
dataset["product"].nunique()

60

In [11]:
print(dataset["product"].unique())

['Traditional Blend Chai ' 'Brazilian ' 'Serenity Green Tea '
 'Our Old Time Diner Blend ' 'Jamaican Coffee River ' 'Ethiopia '
 'English Breakfast ' 'Sustainably Grown Organic ' 'Earl Grey '
 'Cappuccino' 'Espresso shot' 'Cappuccino ' 'Latte' 'Dark chocolate '
 'Columbian Medium Roast ' 'Oatmeal Scone' 'Morning Sunrise Chai '
 'Peppermint ' 'Jumbo Savory Scone' 'Lemon Grass ' 'Latte '
 'Chocolate Chip Biscotti' 'Spicy Eye Opener Chai ' 'Ginger Biscotti'
 'Chocolate Croissant' 'Hazelnut Biscotti' 'Cranberry Scone'
 'Scottish Cream Scone ' 'Croissant' 'Almond Croissant' 'Ginger Scone'
 'Ouro Brasileiro shot' 'Organic Decaf Blend' 'Chocolate syrup'
 'Peppermint' 'Ethiopia' 'Hazelnut syrup' 'Carmel syrup'
 'English Breakfast' 'Sugar Free Vanilla syrup' 'Our Old Time Diner Blend'
 'Jamacian Coffee River' 'Spicy Eye Opener Chai'
 'Sustainably Grown Organic' 'Dark chocolate'
 'Guatemalan Sustainably Grown' 'Civet Cat' 'Serenity Green Tea'
 'Chili Mayan' 'Primo Espresso Roast' 'Brazilian - Or

#### Choose Product Subset

In [12]:
products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ]

In [13]:
dataset = dataset[dataset['product'].isin(products_to_take)]
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot
21,134,2019-04-01,3,189,38,2,Coffee,Latte
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino
23,137,2019-04-01,3,264,40,1,Coffee,Cappuccino


In [14]:
dataset[['product','product_category']].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Cappuccino,Coffee
1,Espresso shot,Coffee
2,Latte,Coffee
3,Oatmeal Scone,Bakery
4,Jumbo Savory Scone,Bakery
5,Chocolate Chip Biscotti,Bakery
6,Ginger Biscotti,Bakery
7,Chocolate Croissant,Bakery
8,Hazelnut Biscotti,Bakery
9,Cranberry Scone,Bakery


#### Clean Transactions

In [15]:
dataset['transaction'] = dataset['transaction_id'].astype(str) +"_"+  dataset['customer_id'].astype(str)
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131
23,137,2019-04-01,3,264,40,1,Coffee,Cappuccino,137_264


In [ ]:
num_of_items_for_each_transaction = dataset['transaction'].value_counts().reset_index()
valid_transactions = num_of_items_for_each_transaction[(num_of_items_for_each_transaction['count']>1)]['transaction'].tolist()
dataset = dataset[dataset['transaction'].isin(valid_transactions)]

In [17]:
num_of_items_for_each_transaction.head()

,transaction,count
0,209_0,31
1,206_0,29
2,204_0,27
3,208_0,25
4,203_0,24


In [19]:
dataset.shape

(7346, 9)

#### Product Trends

In [20]:
dataset["product_category"].value_counts()

product_category
Bakery                3459
Flavours              1972
Coffee                1896
Packaged Chocolate      19
Name: count, dtype: int64

In [21]:
dataset['product'].value_counts()

product
Cappuccino                  645
Latte                       631
Espresso shot               620
Chocolate Croissant         601
Sugar Free Vanilla syrup    530
Chocolate syrup             506
Carmel syrup                486
Hazelnut syrup              450
Ginger Scone                381
Croissant                   330
Jumbo Savory Scone          319
Almond Croissant            316
Chocolate Chip Biscotti     315
Cranberry Scone             308
Hazelnut Biscotti           307
Oatmeal Scone               293
Ginger Biscotti             289
Dark chocolate               19
Name: count, dtype: int64